## Deliverable 2. Create a Customer Travel Destinations Map.

In [47]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [48]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C://Users//12702//World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Bengkulu,ID,-3.8004,102.2655,76.24,85,98,6.67,overcast clouds
1,1,Abu Road,IN,24.4833,72.7833,61.36,50,0,2.51,clear sky
2,2,Narsaq,GL,60.9167,-46.0500,7.02,58,100,3.06,overcast clouds
3,3,Lavrentiya,RU,65.5833,-171.0000,-12.64,93,5,6.20,clear sky
4,4,Banda Aceh,ID,5.5577,95.3222,76.14,92,100,11.63,moderate rain
5,5,Mocuba,MZ,-16.8392,36.9856,72.70,96,32,4.76,light rain
6,6,Manono,CD,-7.3000,27.4167,71.06,93,99,4.09,moderate rain
7,7,Albany,US,42.6001,-73.9662,24.62,51,16,3.00,few clouds
8,8,San Carlos De Bariloche,AR,-41.1456,-71.3082,76.75,23,0,4.61,clear sky
9,9,Adrar,MR,20.5022,-10.0711,83.46,6,0,8.79,clear sky


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
5,5,Mocuba,MZ,-16.8392,36.9856,72.70,96,32,4.76,light rain
6,6,Manono,CD,-7.3000,27.4167,71.06,93,99,4.09,moderate rain
12,12,Punta Arenas,CL,-53.1500,-70.9167,68.11,49,75,17.27,broken clouds
13,13,Sawakin,SD,19.1059,37.3321,71.42,60,4,12.15,clear sky
23,23,Hualmay,PE,-11.0964,-77.6139,74.53,70,97,9.57,overcast clouds
25,25,Saldanha,ZA,-33.0117,17.9442,73.45,92,43,11.32,scattered clouds
28,28,Taiobeiras,BR,-15.8078,-42.2331,72.23,86,100,5.03,moderate rain
29,29,Chuy,UY,-33.6971,-53.4616,70.57,52,75,15.19,broken clouds
31,31,Hilo,US,19.7297,-155.0900,68.79,95,100,9.22,light rain
33,33,Kahului,US,20.8947,-156.4700,73.85,82,0,10.36,clear sky


In [51]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                105
City                   105
Country                104
Lat                    105
Lng                    105
Max Temp               105
Humidity               105
Cloudiness             105
Wind Speed             105
Weather Description    105
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace = True) 

C:\Users\12702\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(5)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
5,Mocuba,MZ,72.70,light rain,-16.8392,36.9856,
6,Manono,CD,71.06,moderate rain,-7.3000,27.4167,
12,Punta Arenas,CL,68.11,broken clouds,-53.1500,-70.9167,
13,Sawakin,SD,71.42,clear sky,19.1059,37.3321,
23,Hualmay,PE,74.53,overcast clouds,-11.0964,-77.6139,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
# Make request and get the JSON data from the search.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
    hotels
        

Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   104
Country                104
Max Temp               104
Weather Description    104
Lat                    104
Lng                    104
Hotel Name             104
dtype: int64

In [59]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [71]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"],["Weather Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))